In [1]:
import os

In [2]:
os.environ['OPENAI_API_KEY'] = os.getenv('Key_OpenAI')
os.environ['SERPER_API_KEY'] = os.getenv('Key_Serper')

In [8]:
from autogen_core.components.models import OpenAIChatCompletionClient, UserMessage

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key=os.getenv['OPENAI_API_KEY'] # Optional if you have an OPENAI_API_KEY env variable set.
)
model_client_result = await model_client.create(
    messages=[
        UserMessage(content="What is the capital of France?", source="user"),
    ]
)
print(model_client_result)  # "Paris"

CreateResult(finish_reason='stop', content='The capital of France is Paris.', usage=RequestUsage(prompt_tokens=15, completion_tokens=7), cached=False, logprobs=None)


In [9]:
model_client_result.content

'The capital of France is Paris.'

In [11]:
import logging

from autogen_agentchat import EVENT_LOGGER_NAME
from autogen_agentchat.agents import ToolUseAssistantAgent
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat.messages import TextMessage
from autogen_core.base import CancellationToken
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(ConsoleLogHandler())
logger.setLevel(logging.INFO)


# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

In [14]:
model_client_result = await model_client.create(
    messages=[
        UserMessage(content="What is the capital of France?", source="user"),
    ]
)
print(model_client_result) 

CreateResult(finish_reason='stop', content='The capital of France is Paris.', usage=RequestUsage(prompt_tokens=15, completion_tokens=7), cached=False, logprobs=None)


/var/folders/wp/6gm0gp7n5653wzt3d3527vsw0000gn/T/ipykernel_3876/281869091.py:1: RuntimeWarning: coroutine 'BaseOpenAIChatCompletionClient.create' was never awaited
  model_client_result = await model_client.create(


In [18]:
async def get_weather(city: str) -> str:
    return f"The weather in {city} is 72 degrees and Sunny."

get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

tool_use_agent = ToolUseAssistantAgent(
    "tool_use_agent",
    system_message="You are a helpful assistant that solves tasks by only using your tools.",
    model_client=model_client,
    registered_tools=[get_weather_tool],
)

tool_result = await tool_use_agent.on_messages(
    messages=[
        TextMessage(content="What is the weather right now in France?", source="user"),
    ],
    cancellation_token=CancellationToken(),
)
print(tool_result)

/var/folders/wp/6gm0gp7n5653wzt3d3527vsw0000gn/T/ipykernel_3876/32951612.py:6: DeprecationWarning: ToolUseAssistantAgent is deprecated. Use AssistantAgent instead.
  tool_use_agent = ToolUseAssistantAgent(


Response(chat_message=TextMessage(source='tool_use_agent', model_usage=RequestUsage(prompt_tokens=74, completion_tokens=41), content='France is a large country with diverse climates, so weather conditions can vary significantly between regions. Could you please specify a particular city or region in France for which you would like to know the current weather?'), inner_messages=[])


In [19]:
tool_result.chat_message.content

'France is a large country with diverse climates, so weather conditions can vary significantly between regions. Could you please specify a particular city or region in France for which you would like to know the current weather?'

In [13]:
import logging

from autogen_agentchat import EVENT_LOGGER_NAME
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent
from autogen_agentchat.logging import ConsoleLogHandler
from autogen_agentchat.task import MaxMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat, SelectorGroupChat
from autogen_core.components.models import OpenAIChatCompletionClient
from autogen_core.components.tools import FunctionTool

# Set up a log handler to print logs to the console.
logger = logging.getLogger(EVENT_LOGGER_NAME)
logger.addHandler(ConsoleLogHandler())
logger.setLevel(logging.INFO)


# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

writing_assistant_agent = CodingAssistantAgent(
    name="writing_assistant_agent",
    system_message="You are a helpful assistant that solve tasks by generating text responses and code.",
    model_client=model_client,
)


async def get_weather(city: str) -> str:
    return f"The weather in {city} is 72 degrees and Sunny."


get_weather_tool = FunctionTool(get_weather, description="Get the weather for a city")

tool_use_agent = ToolUseAssistantAgent(
    "tool_use_agent",
    system_message="You are a helpful assistant that solves tasks by only using your tools.",
    model_client=model_client,
    registered_tools=[get_weather_tool],
)

/var/folders/wp/6gm0gp7n5653wzt3d3527vsw0000gn/T/ipykernel_7718/3560099154.py:23: DeprecationWarning: CodingAssistantAgent is deprecated. Use AssistantAgent instead.
  writing_assistant_agent = CodingAssistantAgent(
/var/folders/wp/6gm0gp7n5653wzt3d3527vsw0000gn/T/ipykernel_7718/3560099154.py:36: DeprecationWarning: ToolUseAssistantAgent is deprecated. Use AssistantAgent instead.
  tool_use_agent = ToolUseAssistantAgent(


In [14]:
termination = MaxMessageTermination(max_messages=2)
round_robin_team = RoundRobinGroupChat([tool_use_agent, writing_assistant_agent], termination_condition=termination)
round_robin_team_result = await round_robin_team.run("Write a Haiku about the weather in Paris")


--------------------------------------------------------------------------- 
[2024-11-02T22:07:32.881251]:

Write a Haiku about the weather in Paris
--------------------------------------------------------------------------- 
[2024-11-02T22:07:32.881251]:

Write a Haiku about the weather in Paris
--------------------------------------------------------------------------- 
[2024-11-02T22:07:32.881251]:

Write a Haiku about the weather in Paris
--------------------------------------------------------------------------- 
[2024-11-02T22:07:32.881251]:

Write a Haiku about the weather in Paris
--------------------------------------------------------------------------- 
[2024-11-02T22:07:32.881251]:

Write a Haiku about the weather in Paris
--------------------------------------------------------------------------- 
[2024-11-02T22:07:32.881251]:

Write a Haiku about the weather in Paris
--------------------------------------------------------------------------- 
[2024-11-02T22:07:34.237212]

In [10]:
termination = MaxMessageTermination(max_messages=2)
llm_team = SelectorGroupChat(
    [tool_use_agent, writing_assistant_agent], model_client=model_client, termination_condition=termination
)

llm_team_result = await llm_team.run("What is the weather in paris right now? Also write a haiku about it.")


--------------------------------------------------------------------------- 
[2024-11-02T22:06:09.823719]:

What is the weather in paris right now? Also write a haiku about it.
--------------------------------------------------------------------------- 
[2024-11-02T22:06:09.823719]:

What is the weather in paris right now? Also write a haiku about it.
--------------------------------------------------------------------------- 
[2024-11-02T22:06:09.823719]:

What is the weather in paris right now? Also write a haiku about it.
--------------------------------------------------------------------------- 
[2024-11-02T22:06:11.084482], tool_use_agent:

The weather in Paris is currently 72 degrees and sunny. 

Here's a Haiku for you:

Gentle sunlit streets,  
Paris basks in golden light,  
Seventy-two's grace.  
--------------------------------------------------------------------------- 
[2024-11-02T22:06:11.084482], tool_use_agent:

The weather in Paris is currently 72 degrees and sunny. 

